# OPEC PDF Extraction and DataFrame Creation

This notebook extracts text from multiple OPEC PDF files, parses the data, and creates a DataFrame with countries as rows and years as columns, then exports the result to CSV.

## 1. Import Required Libraries
Import libraries such as PyPDF2 or pdfplumber for PDF extraction, pandas for data manipulation, and os for file handling.

In [1]:
# Import required libraries
import os
import pandas as pd
import pdfplumber

## 2. Load and Extract Text from PDF Files

Iterate through the specified PDF files (24.pdf, 19-23.pdf, 14-18.pdf), extract text from each, and store the raw text for further processing.

In [2]:
# List of PDF files to extract
pdf_files = ['24.pdf', '19-23.pdf', '14-18.pdf']
raw_texts = {}

for pdf_file in pdf_files:
    with pdfplumber.open(pdf_file) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text() + '\n'
        raw_texts[pdf_file] = text

# Show a preview of the extracted text for each file
for fname, text in raw_texts.items():
    print(f'--- {fname} ---')
    print(text[:1000])  # Print first 1000 characters
    print('\n')

--- 24.pdf ---
Oil data: upstream
Table 3.5
World crude oil production by country (1,000 b/d)
change
2020 2021 2022 2023 2024
24/23
OECD Americas 14,186 14,160 14,848 15,817 16,013 196
Canada 1,199 1,185 1,231 1,227 1,249 23
Chile 1 2 2 2 2 0
Mexico 1,663 1,665 1,622 1,653 1,553 –100
United States 11,323 11,308 11,992 12,935 13,208 274
OECD Europe 2,969 2,898 2,724 2,753 2,685 –68
Denmark 71 66 65 58 60 2
Norway 1,691 1,764 1,685 1,794 1,768 –26
United Kingdom 931 792 724 645 579 –67
Others 277 276 251 255 278 23
OECD Asia Pacific 163 143 129 106 95 –12
Australia 138 121 110 85 77 –9
Others 25 22 19 21 18 –3
China 3,889 3,988 4,094 4,181 4,244 62
India 587 574 563 544 533 –12
Other Asia 1,670 1,542 1,431 1,375 1,324 –52
Brunei 85 83 71 67 75 8
Indonesia 631 585 539 540 517 –23
Malaysia 456 418 397 373 341 –31
Thailand 117 98 79 70 81 11
Vietnam 194 184 180 173 162 –11
Others 188 175 166 153 148 –5
Latin America 5,536 5,575 6,026 6,642 7,014 372
Argentina 480 513 582 635 701 65
Brazil 2

## 3. Parse Extracted Text into Structured Data

Process the raw text to extract country names and their corresponding data for each year, organizing the information into a structured format suitable for a DataFrame.

In [10]:
# Updated parsing logic: extract from each PDF separately and only the relevant years
import re
from collections import defaultdict

# Define which years to extract from each file
pdf_years = {
    '24.pdf': ['2024'],
    '19-23.pdf': ['2019', '2020', '2021', '2022', '2023'],
    '14-18.pdf': ['2014', '2015', '2016', '2017', '2018']
}

# Helper function to extract table-like data from text
# This function assumes each line is: Country value1 value2 ... valueN
# and the order of years matches pdf_years
country_data = defaultdict(dict)

for pdf_file, years in pdf_years.items():
    text = raw_texts.get(pdf_file, '')
    lines = text.split('\n')
    for line in lines:
        # Skip empty or non-data lines
        if not line.strip():
            continue
        # Try to match: country name (possibly with spaces), then N numbers
        match = re.match(r'^(\D+?)\s+([\d\.,\s]+)$', line)
        if match:
            country = match.group(1).strip()
            values = re.findall(r'[\d\.,]+', match.group(2))
            if len(values) == len(years):
                for y, v in zip(years, values):
                    country_data[country][y] = v

# Create DataFrame: countries as rows, years as columns
all_years = sum(pdf_years.values(), [])
df = pd.DataFrame.from_dict(country_data, orient='index', columns=all_years)
df.index.name = 'Country'
df.columns.name = 'Year'
df.head()

Year,2024,2019,2020,2021,2022,2023,2014,2015,2016,2017,2018
Country,,,,,,,,,,,
Table,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OPEC percentage,NaN,37.4,35.5,36.4,38.1,36.6,41.7,42.1,43.9,43.1,41.9


## 4. Create DataFrame with Countries as Observations and Years as Columns

Use pandas to create a DataFrame where each row represents a country and each column represents a year, filling in the extracted data.

In [11]:
# Clean up DataFrame: ensure countries as rows, years as columns
# This assumes 'parsed_df' from previous cell

df = parsed_df.copy()
df.index.name = 'Country'
df.columns.name = 'Year'
df.head()

Year,2000
Country,
Table,3.5
2020,2021
Chile,6.6
Denmark,71
Others,420.9


## 5. Export DataFrame to CSV

Save the resulting DataFrame to a CSV file for further analysis or sharing.

In [ ]:
# Export DataFrame to CSV
output_csv = 'opec_countries_years.csv'
df.to_csv(output_csv)
print(f'DataFrame exported to {output_csv}')

In [26]:
# Robust parsing for 19-23.csv: clean country names of quotes and whitespace
import pandas as pd
import re

expected_years = ['2019', '2020', '2021', '2022', '2023']
rows = []

def clean_country(name):
    return name.strip().strip('"\'')

with open('19-23.csv', encoding='latin1') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        # Find the position of the first number (with comma or dot)
        first_num = re.search(r'[\d]', line)
        if not first_num:
            continue
        country = clean_country(line[:first_num.start()])
        numbers = re.findall(r'[\d,\.]+', line[first_num.start():])
        if country and len(numbers) >= 5:
            rows.append([country] + numbers[:5])

# Build DataFrame
fixed_19_23 = pd.DataFrame(rows, columns=['Country'] + expected_years)
print(fixed_19_23.head(10))
print(f"Total countries extracted: {len(fixed_19_23)}")

# Save cleaned CSV
fixed_19_23.to_csv('19-23_cleaned.csv', index=False)
print('Cleaned 19-23.csv saved as 19-23_cleaned.csv')

          Country    2019    2020    2021    2022    2023
0   OECD Americas  15,297  14,182  14,119  14,766  15,807
1          Canada   1,304   1,199   1,185   1,231   1,225
2           Chile       3       1       2       2       2
3          Mexico   1,679   1,663   1,665   1,622   1,653
4   United States  12,311  11,318  11,268  11,911  12,927
5     OECD Europe   2,775   2,977   2,904   2,724   2,753
6         Denmark     102      71      66      65      58
7          Norway   1,408   1,699   1,770   1,685   1,794
8  United Kingdom   1,007     931     792     724     645
9          Others     258     277     276     251     255
Total countries extracted: 59
Cleaned 19-23.csv saved as 19-23_cleaned.csv


In [27]:
# --- Robust parsing for 14-18.csv ---
expected_years_1418 = ['2014', '2015', '2016', '2017', '2018']
rows_1418 = []

def clean_country(name):
    return name.strip().strip('"\'')

with open('14-18.csv', encoding='latin1') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        first_num = re.search(r'[\d]', line)
        if not first_num:
            continue
        country = clean_country(line[:first_num.start()])
        numbers = re.findall(r'[\d,\.]+', line[first_num.start():])
        if country and len(numbers) >= 5:
            rows_1418.append([country] + numbers[:5])

fixed_1418 = pd.DataFrame(rows_1418, columns=['Country'] + expected_years_1418)
print(fixed_1418.head(10))
print(f"Total countries extracted (14-18): {len(fixed_1418)}")
fixed_1418.to_csv('14-18_cleaned.csv', index=False)
print('Cleaned 14-18.csv saved as 14-18_cleaned.csv')

# --- Robust parsing for 24.csv ---
expected_years_24 = ['2024']
rows_24 = []

with open('24.csv', encoding='latin1') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        first_num = re.search(r'[\d]', line)
        if not first_num:
            continue
        country = clean_country(line[:first_num.start()])
        numbers = re.findall(r'[\d,\.]+', line[first_num.start():])
        if country and len(numbers) >= 1:
            rows_24.append([country] + numbers[:1])

fixed_24 = pd.DataFrame(rows_24, columns=['Country'] + expected_years_24)
print(fixed_24.head(10))
print(f"Total countries extracted (24): {len(fixed_24)}")
fixed_24.to_csv('24_cleaned.csv', index=False)
print('Cleaned 24.csv saved as 24_cleaned.csv')

         Country      2014      2015      2016      2017      2018
0  North America  10,158.6  10,694.2  10,016.1  10,564.4  12,237.7
1         Canada   1,399.8   1,263.4   1,185.5   1,212.8   1,276.0
2  United States   8,758.7   9,430.8   8,830.6   9,351.6  10,961.7
3  Latin America   9,740.7   9,714.0   9,218.7   8,703.6   8,006.6
4      Argentina     532.2     532.3     510.6     479.5     489.4
5         Brazil   2,254.6   2,437.3   2,510.0   2,621.8   2,586.5
6          Chile       6.6       4.5       3.7       3.0       3.0
7       Colombia     990.3   1,005.6     883.3     853.6     865.2
8        Ecuador     556.6     543.1     549.0     531.3     517.2
9         Mexico   2,428.9   2,266.8   2,153.5   1,948.4   1,813.0
Total countries extracted (14-18): 64
Cleaned 14-18.csv saved as 14-18_cleaned.csv
          Country    2024
0   OECD Americas  14,186
1          Canada   1,199
2           Chile       1
3          Mexico   1,663
4   United States  11,323
5     OECD Europe   2,96

In [20]:
# --- Robust parsing for 24.csv (handle Kuwait and Saudi Arabia special cases, others use last number) ---
expected_years_24 = ['2024']
rows_24 = []
import re

def clean_country(name):
    # Remove trailing digits (e.g., 'Kuwait1' -> 'Kuwait')
    return re.sub(r'\d+$', '', name.strip().strip('"\''))

with open('24.csv', encoding='latin1') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        first_num = re.search(r'[\d]', line)
        if not first_num:
            continue
        country_raw = line[:first_num.start()]
        numbers = re.findall(r'[\d,\.]+', line[first_num.start():])
        country = clean_country(country_raw)
        # Special handling for Kuwait and Saudi Arabia, others use last number
        if country.lower() == 'kuwait' and len(numbers) >= 7:
            value = numbers[5]
        elif country.lower() in ['saudi arabia', 'saudiarabia'] and len(numbers) >= 6:
            value = numbers[5]
        elif len(numbers) >= 1:
            value = numbers[-2]
        else:
            continue
        rows_24.append([country, value])

fixed_24 = pd.DataFrame(rows_24, columns=['Country'] + expected_years_24)
print(fixed_24.head(10))
print(f"Total countries extracted (24): {len(fixed_24)}")
fixed_24.to_csv('24_cleaned.csv', index=False)
print('Cleaned 24.csv saved as 24_cleaned.csv')

          Country    2024
0                      24
1   OECD Americas  16,013
2          Canada   1,249
3           Chile       2
4          Mexico   1,553
5   United States  13,208
6     OECD Europe   2,685
7         Denmark      60
8          Norway   1,768
9  United Kingdom     579
Total countries extracted (24): 61
Cleaned 24.csv saved as 24_cleaned.csv


In [23]:
# Filter each cleaned CSV to only the specified OPEC countries and save as *_opec_only.csv
opec_countries = [
    'Iran', 'Iraq', 'Kuwait', 'Saudi Arabia', 'Venezuela', 'Libya', 'United Arab Emirates',
    'Algeria', 'Nigeria', 'Qatar', 'Ecuador', 'Angola', 'Gabon', 'Equatorial Guinea', 'Congo', 'Indonesia'
 ]

def normalize_country(name):
    return str(name).strip().lower().replace('’', "'")

opec_countries_norm = [normalize_country(c) for c in opec_countries]

def filter_opec(df):
    df['Country_norm'] = df['Country'].apply(normalize_country)
    filtered = df[df['Country_norm'].isin(opec_countries_norm)].copy()
    filtered.drop(columns=['Country_norm'], inplace=True)
    return filtered

# Process each cleaned CSV
df_1418 = pd.read_csv('14-18_cleaned.csv')
df_1923 = pd.read_csv('19-23_cleaned.csv')
df_24 = pd.read_csv('24_cleaned.csv')

df_1418_opec = filter_opec(df_1418)
df_1923_opec = filter_opec(df_1923)
df_24_opec = filter_opec(df_24)

df_1418_opec.to_csv('14-18_cleaned_opec_only.csv', index=False)
df_1923_opec.to_csv('19-23_cleaned_opec_only.csv', index=False)
df_24_opec.to_csv('24_cleaned_opec_only.csv', index=False)
print('Filtered OPEC-only CSVs saved as *_opec_only.csv')
print('14-18:', df_1418_opec['Country'].tolist())
print('19-23:', df_1923_opec['Country'].tolist())
print('24:', df_24_opec['Country'].tolist())

Filtered OPEC-only CSVs saved as *_opec_only.csv
14-18: ['Ecuador', 'Venezuela', 'Iraq', 'Kuwait', 'Qatar', 'Saudi Arabia', 'United Arab Emirates', 'Algeria', 'Angola', 'Congo', 'Equatorial Guinea', 'Gabon', 'Libya', 'Nigeria', 'Indonesia']
19-23: ['Indonesia', 'Ecuador', 'Venezuela', 'Iraq', 'Kuwait', 'Qatar', 'Saudi Arabia', 'United Arab Emirates', 'Algeria', 'Angola', 'Congo', 'Equatorial Guinea', 'Gabon', 'Libya', 'Nigeria']
24: ['Indonesia', 'Ecuador', 'Venezuela', 'Iraq', 'Kuwait', 'Qatar', 'Saudi Arabia', 'United Arab Emirates', 'Algeria', 'Angola', 'Congo', 'Equatorial Guinea', 'Gabon', 'Libya', 'Nigeria']


In [25]:
# Merge all OPEC-only cleaned CSVs into a single DataFrame (2014-2024)
import pandas as pd

# Read filtered OPEC-only CSVs
df_1418 = pd.read_csv('14-18_cleaned_opec_only.csv')
df_1923 = pd.read_csv('19-23_cleaned_opec_only.csv')
df_24 = pd.read_csv('24_cleaned_opec_only.csv')

# Merge on 'Country' (outer join to keep all specified OPEC countries)
merged = df_1418.merge(df_1923, on='Country', how='outer').merge(df_24, on='Country', how='outer')

# Optional: sort columns by year
cols = ['Country'] + sorted([c for c in merged.columns if c != 'Country'])
merged = merged[cols]

# Save to CSV
merged.to_csv('merged_opec_2014_2024_opec.csv', index=False)
print('Merged OPEC-only CSV saved as merged_opec_2014_2024_opec.csv')
print(merged.head(10))

Merged OPEC-only CSV saved as merged_opec_2014_2024_opec.csv
             Country     2014     2015     2016     2017     2018   2019  \
0            Algeria  1,192.8  1,157.1  1,146.3  1,058.7  1,040.1  1,023   
1             Angola  1,653.7  1,767.1  1,721.6  1,632.2  1,473.3  1,373   
2              Congo    240.6    232.2    225.0    262.7    323.5    329   
3            Ecuador    556.6    543.1    549.0    531.3    517.2    531   
4  Equatorial Guinea    199.8    185.3    160.1    128.6    120.2    110   
5              Gabon    210.9    213.5    220.7    210.1    193.4    218   
6          Indonesia    697.3    690.1    737.9    710.2    684.4    664   
7               Iraq  3,110.5  3,504.1  4,647.8  4,468.7  4,410.0  4,576   
8             Kuwait        1  2,866.8  2,858.7  2,954.3  2,704.2      1   
9              Libya    479.9    403.7    389.1    810.5    951.2  1,091   

    2020   2021   2022   2023   2024  
0    899    911  1,020    973    907  
1  1,271  1,124  1,137  